In [2]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
import numpy as np
import pickle
import json
import glob
import os

import matplotlib.pyplot as plt
%matplotlib inline


from enterprise.signals import signal_base
from enterprise.signals import gp_signals
from enterprise.pulsar import Pulsar
from enterprise.signals import parameter
from enterprise.signals import gp_priors

from enterprise_extensions import model_utils, blocks
from enterprise_extensions import model_orfs
from enterprise_extensions.frequentist import optimal_statistic as opt_stat
from enterprise.signals import selections

from enterprise_warp import enterprise_warp
#from enterprise_warp import enterprise_warp

import h5py
import la_forge.core as co

In [3]:
with open('/Users/physics/nanogw/data/ppta_dr2.pkl','rb') as fin:
    psrs=pickle.load(fin)
print(len(psrs))

26


In [11]:
noisefile = '/Users/physics/Desktop/default_noise_example_1.json'

with open(noisefile, 'r') as f:
    noisedict = json.load(f)

noisefile2 ='/Users/physics/Desktop/default_noise_example_2.json'
with open(noisefile2, 'r') as f:
    noisedict2 = json.load(f)

In [12]:
noisedict

{'model_name': 'examp_1',
 'universal': {'white_noise': 'by_backend',
  'spin_noise': 'powerlaw',
  'dm_noise': 'powerlaw'},
 'common_signals': {},
 'J1832-0836': {'efac': 'by_backend',
  'equad': 'by_backend',
  'spin_noise': 'powerlaw',
  'dm_noise': 'powerlaw'}}

In [13]:
noisedict2

{'model_name': 'examp_2',
 'universal': {'white_noise': 'by_backend',
  'spin_noise': 'powerlaw',
  'dm_noise': 'powerlaw'},
 'common_signals': {},
 'J1832-0836': {'efac': 'by_backend',
  'spin_noise': 'turnover',
  'dm_noise': 'powerlaw'}}

In [14]:
noisefile3 = '//Users/physics/nanogw/data/ppta_dr2_TN_noise.json'

with open(noisefile3, 'r') as f:
    noisedict3 = json.load(f)

In [15]:
noisedict3

{'J0437-4715_0_magnexp__idx': 0.4257147497466019,
 'J0437-4715_0_magnexp__log10_Amp': -6.302840431334452,
 'J0437-4715_0_magnexp__log10_tau': 2.1762989491448765,
 'J0437-4715_0_magnexp__t0': 57080.77195329371,
 'J0437-4715_CASPSR_40CM_efac': 0.7430934464857502,
 'J0437-4715_CASPSR_40CM_log10_tnequad': -8.20890044773843,
 'J0437-4715_CPSR2_10CM_efac': 1.1394645851791254,
 'J0437-4715_CPSR2_10CM_log10_tnequad': -6.602670339806825,
 'J0437-4715_CPSR2_20CM_efac': 2.3071643789004312,
 'J0437-4715_CPSR2_20CM_log10_tnequad': -6.7928773256385755,
 'J0437-4715_CPSR2_50CM_efac': 1.120916497680611,
 'J0437-4715_CPSR2_50CM_log10_tnequad': -8.499957994720189,
 'J0437-4715_PDFB1_10CM_efac': 0.72575092805256,
 'J0437-4715_PDFB1_10CM_log10_tnequad': -6.6652497106773065,
 'J0437-4715_PDFB1_1433_efac': 3.339282613815889,
 'J0437-4715_PDFB1_1433_log10_tnequad': -8.422054371246592,
 'J0437-4715_PDFB1_20CM_efac': 2.640138126930894,
 'J0437-4715_PDFB1_20CM_log10_tnequad': -8.219148134553391,
 'J0437-4715_PD

In [18]:
class opts(object):pass
setattr(opts,'extra_model_terms',None)
setattr(opts,'job_config_xlsx',None)
setattr(opts,'num',18)
setattr(opts,'mpi_regime',None)
setattr(opts,'wipe_old_output',None)

In [19]:
params=enterprise_warp.Params('/Users/physics/Desktop/default_hypermodel.txt',opts=opts)

------------------
Setting default parameters with file  /Users/physics/Desktop/default_hypermodel.txt
Setting default Solar System Ephemeris: DE436
Setting a default Enterprise clock convention (check the code)
Using all available pulsars from .par/.tim directory
Setting a default linear timing model
Including transient events to specific pulsar models
Setting reference radio frequency to 1400 MHz
------------------
Setting sampler kwargs from the parameter file:
Setting SCAMweight to 30
Setting AMweight to 15
Setting DEweight to 50
------------------
Current .par file:  J1832-0836.par
Current .tim file:  J1832-0836.tim


In [6]:
j1832_m =enterprise_warp.StandardModels(psrs[18],params)

In [20]:
dm=j1832_m.dm_noise()
efac=j1832_m.efac()

In [24]:
j1832_m.equad()

NotImplementedError: EquadNoise was removed in enterprise v3.3. Use MeasurementNoise to implement tempo/tempo2/pint definition [efac^2 (toaerr^2 + t2equad^2)] or TNEquadNoise to obtain legacy enterprise definition for EQUAD only [tnequad^2].

In [21]:
model=dm+efac+ecorr+spin_noise

In [9]:
#Problem Children 
j1832_m.system_noise()
#j1832_m.ppta_band_noise()

UnboundLocalError: local variable 'syn' referenced before assignment

In [ ]:
%%time

# find the maximum time span to set GW frequency sampling
Tspan = model_utils.get_tspan(psrs)

# Here we build the signal model
# First we add the timing model
s = gp_signals.TimingModel()


# Then we add the white noise
# There are three types of white noise: EFAC, EQUAD, 
# We use different white noise parameters for every backend/receiver combination
# The white noise parameters are held constant
#inc_ecorr======FALSE because ppta
s += blocks.white_noise_block(vary=False, inc_ecorr=False, select='backend', tnequad=True)

#DMGP
dm = blocks.dm_noise_block(gp_kernel="diag", psd="powerlaw", components=100, Tspan=Tspan)

# Next comes the individual pulsar red noise
# We model the red noise as a Fourier series with 30 frequency components, 
# with a power-law PSD
s += blocks.red_noise_block(prior='log-uniform', Tspan=Tspan, components=30)

# Finally, we add the common red noise, which is modeled as a Fourier series with 5 frequency components
# The common red noise has a power-law PSD with spectral index of 4.33
s += blocks.common_red_noise_block(psd='powerlaw', prior='log-uniform', Tspan=Tspan, 
                                   components=5, gamma_val=4.33, name='gw')

models=[]
for p in psrs:
    if p.name not in ['J0437-4715_ppta', 'J1713+0747_ppta', 'J0613-0200_ppta','J1017-7156_ppta','J1045-4509_ppta']:
        models.append((s+dm)(p))
    elif p.name=='J0437-4715_ppta':
#         models.append(s(p))
        models.append((s+dm+sn+bn)(p))
    elif p.name=='J1713+0747_ppta':
        models.append(s(p))
#         models.append((s+dm+j1713dips)(p))
    elif p.name in [ 'J0613-0200_ppta','J1017-7156_ppta','J1045-4509_ppta']:
        models.append(s(p))
# We set up the PTA object using the signal we defined above and the pulsars





'''
import cloudpickle
with open('/Users/physics/nanogw/data/ppta_models.pkl','wb') as fout:
    cloudpickle.dump(models,fout)
'''


pta = signal_base.PTA(models)

# We need to set the white noise parameters to the values in the noise dictionary
pta.set_default_params(noisedictcp2)

ostat = opt_stat.OptimalStatistic(psrs, pta=pta, orf='hd')
ostat_dip = opt_stat.OptimalStatistic(psrs, pta=pta, orf='dipole')
ostat_mono = opt_stat.OptimalStatistic(psrs, pta=pta, orf='monopole')

In [ ]:
"""
#Code to generate and save max likelihood parameters only need to run once
#To get the files generated 
c0 = co.Core(chaindir='/Users/physics/nanogw/data/chains_ppta_dr2/')
max_like=c0.get_map_dict()
# create a binary pickle file 
f = open("/Users/physics/nanogw/data/ppta_ml_params.pkl","wb")
# write the python object (dict) to pickle file
pickle.dump(max_like,f)
# close file
f.close()
"""

In [ ]:
c0 = co.Core(chaindir='/Users/physics/nanogw/data/chains_ppta_dr2/')

In [ ]:
c0.params

In [ ]:
# Load up the maximum-likelihood values for the pulsars' red noise parameters and the common red process
# These values come from the results of a Bayesian search

with open("/Users/physics/nanogw/data/ppta_ml_params.pkl",'rb') as fin:
    ml_params=pickle.load(fin)

In [ ]:
ml_paramscp=ml_params.copy()
for ky,val in ml_params.items():
    split=ky.split('_')
    split.insert(1,'ppta')
    kynew='_'.join(split)
    ml_paramscp[kynew] = ml_params[ky]
for ky,val in ml_params.items():
    if ky in ml_params:
        ml_paramscp.pop(ky)
ml_paramscp['gw_log10_A']=ml_paramscp['gw_ppta_log10_A']

In [ ]:
ml_params

In [ ]:
'''
#Run to save renamed files for ease
ml=ml_paramscp
# create a binary pickle file 
f = open("/Users/physics/nanogw/data/ppta_ml_params_renamed.pkl","wb")
# write the python object (dict) to pickle file
pickle.dump(ml,f)
# close file
f.close()
'''

In [ ]:
# Compute the optimal statistic
# The optimal statistic returns five quantities:
#  - xi: an array of the angular separations between the pulsar pairs (in radians)
#  - rho: an array of the cross-correlations between the pulsar pairs
#  - sig: an array of the uncertainty in the cross-correlations
#  - OS: the value of the optimal statistic
#  - OS_sig: the uncertainty in the optimal statistic

xi, rho, sig, OS, OS_sig = ostat.compute_os(params=ml_paramscp) #HD
print(OS, OS_sig, OS/OS_sig)

_, _, _, OS_dip, OS_sig_dip = ostat_dip.compute_os(params=ml_paramscp) #Dipole
print(OS_dip, OS_sig_dip, OS_dip/OS_sig_dip)

_, _, _, OS_mono, OS_sig_mono = ostat_mono.compute_os(params=ml_paramscp) #Monopole 
print(OS_mono, OS_sig_mono, OS_mono/OS_sig_mono)

In [ ]:
# Plot the cross-correlations and compare to the Hellings-Downs curve
# Before plotting, we need to bin the cross-correlations

def weightedavg(rho, sig):
    weights, avg = 0., 0.
    for r,s in zip(rho,sig):
        weights += 1./(s*s)
        avg += r/(s*s)
        
    return avg/weights, np.sqrt(1./weights)

def bin_crosscorr(zeta, xi, rho, sig):
    
    rho_avg, sig_avg = np.zeros(len(zeta)), np.zeros(len(zeta))
    
    for i,z in enumerate(zeta[:-1]):
        myrhos, mysigs = [], []
        for x,r,s in zip(xi,rho,sig):
            if x >= z and x < (z+10.):
                myrhos.append(r)
                mysigs.append(s)
        rho_avg[i], sig_avg[i] = weightedavg(myrhos, mysigs)
        
    return rho_avg, sig_avg

# sort the cross-correlations by xi
idx = np.argsort(xi)

xi_sorted = xi[idx]
rho_sorted = rho[idx]
sig_sorted = sig[idx]

# bin the cross-correlations so that there are the same number of pairs per bin
#number of pulsar correlations you want per bin
npairs = 65

xi_mean = []
xi_err = []

rho_avg = []
sig_avg = []

i = 0

while i < len(xi_sorted):
    
    xi_mean.append(np.mean(xi_sorted[i:npairs+i]))
    xi_err.append(np.std(xi_sorted[i:npairs+i]))

    r, s = weightedavg(rho_sorted[i:npairs+i], sig_sorted[i:npairs+i])
    rho_avg.append(r)
    sig_avg.append(s)
    
    i += npairs
    
xi_mean = np.array(xi_mean)
xi_err = np.array(xi_err)

In [ ]:
def get_HD_curve(zeta):
    
    coszeta = np.cos(zeta*np.pi/180.)
    xip = (1.-coszeta) / 2.
    HD = 3.*( 1./3. + xip * ( np.log(xip) -1./6.) )
    
    return HD/2

In [ ]:
(_, caps, _) = plt.errorbar(xi_mean*180/np.pi, rho_avg, xerr=xi_err*180/np.pi, yerr=sig_avg, marker='o', ls='', 
                            color='0.1', fmt='o', capsize=4, elinewidth=1.2)

zeta = np.linspace(0.01,180,100)
HD = get_HD_curve(zeta+1)

plt.plot(zeta, OS*HD, ls='--', label='Hellings-Downs', color='C0', lw=1.5)
plt.plot(zeta, zeta*0.0+OS_mono, ls='--', label='Monopole', color='C1', lw=1.5)
plt.plot(zeta, OS_dip*np.cos(zeta*np.pi/180), ls='--', label='Dipole', color='C2', lw=1.5)

plt.xlim(0, 180);
#plt.ylim(-3.5e-29, 3.5e-29);
plt.ylabel(r'$\hat{A}^2 \Gamma_{ab}(\zeta)$')
plt.xlabel(r'$\zeta$ (deg)');

plt.legend(loc=4);

plt.tight_layout();
plt.show();

In [ ]:
def hd_orf(pos1, pos2, diag=1.):
    """Hellings & Downs spatial correlation function."""
    if np.all(np.isclose(pos1,pos2,rtol=1e-05, atol=1e-08)):
        return diag
    else:
        omc2 = (1 - np.dot(pos1, pos2)) / 2
        return 1.5 * omc2 * np.log(omc2) - 0.25 * omc2 + 0.5

In [ ]:
zeta = np.linspace(0.01,180,100)
HD = get_HD_curve(zeta+1)

plt.plot(zeta, OS*HD, ls='--', label='Hellings-Downs', color='C0', lw=1.5)
plt.plot(zeta, zeta*0.0+OS_mono, ls='--', label='Monopole', color='C1', lw=1.5)
plt.plot(zeta, OS_dip*np.cos(zeta*np.pi/180), ls='--', label='Dipole', color='C2', lw=1.5)
plt.xlim(0, 180);
plt.ylim(-1.5e-29, 1.5e-29);
plt.ylabel(r'$\hat{A}^2 \Gamma_{ab}(\zeta)$')
plt.xlabel(r'$\zeta$ (deg)');